In [1]:
import numpy as np
import pImpactR as impact
from copy import deepcopy as copy
import matplotlib.pyplot as plt
import sys
np.set_printoptions(formatter={'float': lambda x: format(x, '1.2E')})

In [2]:
NTURN = [1024,1024*2]

nCore_y = 4
nCore_z = 4

NL_c = 0.01
NL_t = -0.4
NL_nu= 0.3
NL_L = 1.8
dTune = 0.0

alfx = np.tan(np.pi*NL_nu)
betx = NL_L/np.sin(2.0*np.pi*NL_nu)

pipe_radius = 0.02*np.sqrt(2.0)

# 1. Preparation

### 1.1 read InputFile and simplify lattice

In [3]:
beam, lattice = impact.readInputFile('test_toymodel.in');
beam.nCore_y = nCore_y
beam.nCore_z = nCore_z
beam.distribution.distribution_type = 'ReadFile'
beam.current = 0.0

ke = beam.kinetic_energy
mass = beam.mass
freq = beam.frequency

reading ImpactZ input file (test_toymodel.in)
  : mpi task info .............................done
  : simulation control parameters .............done
  : space charge field solver, mesh info ......done
  : dist-type,restart,subcycle,#of state ......done
  : Multiple Charge State info ................done
  : particle distribution info ................done
  : beam reference orbit info .................done
  : converting impact dist to twiss param......done
  : lattice info ..............................done


In [4]:
def normalize(data):
    data[:,1] = (data[:,0]*alfx/np.sqrt(betx) + data[:,1]*np.sqrt(betx))/NL_c
    data[:,3] = (data[:,2]*alfx/np.sqrt(betx) + data[:,3]*np.sqrt(betx))/NL_c
    data[:,0] = data[:,0]/(np.sqrt(betx)*NL_c)
    data[:,2] = data[:,2]/(np.sqrt(betx)*NL_c)
    
def unnormalize(data):
    data[:,1] = (-data[:,0]*alfx + data[:,1])*NL_c/np.sqrt(betx)
    data[:,3] = (-data[:,2]*alfx + data[:,3])*NL_c/np.sqrt(betx)
    data[:,0] = data[:,0]*np.sqrt(betx)*NL_c
    data[:,2] = data[:,2]*np.sqrt(betx)*NL_c

In [5]:
for i in range(len(lattice)-1,0,-1):
    if lattice[i].type == 'write_raw_ptcl':
        lattice.pop(i)
lattice[1].n_sckick = 100
lattice[1].length = NL_L
lattice[1].strength_t = NL_t
lattice[1].transverse_scale_c = NL_c
lattice[1].tune_advance = NL_nu
lattice[1].pipe_radius = 0.02*np.sqrt(2)

lattice[2].tune_advance = dTune
lattice[2].nonlinear_insert_tuneAdvance = NL_nu
lattice[2].nonlinear_insert_length = NL_L

### 1.2. generate particles

In [6]:
testP = 0.0
k=0
q_m = beam.multi_charge.q_m[0]
pTest = []
nx = 100
xmax = 0.98
for x in np.linspace(-xmax,xmax,nx):
    for y in np.linspace(-xmax,xmax,nx):
        if x**2+y**2<=xmax**2 :
            k+=1
            pTest.append([x,0.0,y,0.0,0.0,0.0,q_m,0.0,k]) 
pTest = np.array(pTest)
npt = len(pTest)
beam.n_particles = npt
print('initial npt=',npt)

initial npt= 7668


In [7]:
unnormalize(pTest)
impact.writeParticleData(pTest, ke, mass, freq)

In [8]:
lattice[0].turns = NTURN[0]

TBT = impact.getElem('TBT')
TBT.file_id = NTURN[0]
TBT.pID_begin = 1
TBT.pID_end = npt
lattice.insert(1,TBT)

OUTPUT = impact.getElem('write_raw_ptcl')
OUTPUT.file_id = NTURN[0]
OUTPUT.turn = NTURN[0]
OUTPUT.format_id = 2
lattice.append(OUTPUT)

# FMA

### Run Forward 

In [9]:
impact.writeInputFile(beam,lattice) 
impact.run(beam)

256

In [ ]:
for nturn in NTURN[:-1]:
    lattice[0].turns = nturn
    TBT.file_id = nturn*2
    OUTPUT.turn = nturn
    OUTPUT.file_id = nturn*2
    beam.distribution.distribution_type = 'ReadFile_binary'
    beam.distribution.file_id = nturn
    impact.writeInputFile(beam,lattice) 
    impact.run(beam)

### Load Data

In [ ]:
iTBT = {}
TBT = {}
X = {}
Y = {}
Npt = {}

for nturn in NTURN:
    print(nturn)
    iTBT[nturn],TBT[nturn] = impact.readTBT(nturn,ke,mass,freq)
    TBT[nturn] = TBT[nturn][:,:4,:]
    Npt[nturn] = len(iTBT[nturn])

In [ ]:
for i in range(len(NTURN)):
    nturn = NTURN[i]
    
    npt = len(iTBT[nturn])
    X[nturn]=np.zeros((nturn,npt),dtype=np.complex)
    Y[nturn]=np.zeros((nturn,npt),dtype=np.complex)
    
    iturn = 0
    for j in range(i+1):
        jturn = NTURN[j]
        
        itmp = np.in1d(iTBT[jturn],iTBT[nturn])
        tmp = TBT[jturn][:,:,itmp]
        X[nturn][iturn:jturn] = (tmp[:,0,:] - 1j*(betx*tmp[:,1,:]+alfx*tmp[:,0,:]))/(NL_c*np.sqrt(betx))
        Y[nturn][iturn:jturn] = (tmp[:,2,:] - 1j*(betx*tmp[:,3,:]+alfx*tmp[:,2,:]))/(NL_c*np.sqrt(betx))
        print(iturn,jturn,nturn,np.sum(itmp),npt)
        
        iturn = jturn

### Apply NAFF

In [ ]:
def getClosestDiff(ref,arr):
    # ref.shape = nSample
    nMode = len(arr)
    if ref>1.0:
        ref = ref - 1.0
    for i in range(nMode):
        if arr[i]>1.0:
            arr[i]=arr[i]-1.0
    diff = np.abs(ref-arr)
    for i in range(nMode):
        if (diff[i]>0.5):
            diff[i] = 1.0 - diff[i]
    return np.min(diff)

In [ ]:
iMax = {}
iMin = {}
tuneXbefore= {}
tuneYbefore= {}
Xdiff = {}
Ydiff = {}
Adiff = {}

for nturn in NTURN[1:]:
    npt = len(X[nturn][0,:])
    tuneXbefore[nturn] = np.zeros(npt)
    tuneYbefore[nturn] = np.zeros(npt)
    Xdiff[nturn] = np.zeros(npt)
    Ydiff[nturn] = np.zeros(npt)
    Adiff[nturn] = np.zeros(npt)
    for i in range(npt):
        signal = X[nturn][:int(nturn/2),i] -np.mean(X[nturn][:int(nturn/2),i])
        tuneXbefore[nturn][i],amp,dummy = impact.util.naff(1,signal,window_id=1)
        signal = X[nturn][int(nturn/2):,i] -np.mean(X[nturn][int(nturn/2):,i])
        tune,amp,dummy = impact.util.naff(3,signal,window_id=1)
        Xdiff[nturn][i] = getClosestDiff(tuneXbefore[nturn][i],tune)
        
        signal = Y[nturn][:int(nturn/2),i] -np.mean(Y[nturn][:int(nturn/2),i])
        tuneYbefore[nturn][i],amp,dummy = impact.util.naff(1,signal,window_id=1)
        signal = Y[nturn][int(nturn/2):,i] -np.mean(Y[nturn][int(nturn/2):,i])
        tune,amp,dummy = impact.util.naff(3,signal,window_id=1)
        Ydiff[nturn][i] = getClosestDiff(tuneYbefore[nturn][i],tune)
        
    Adiff[nturn] = np.sqrt(Xdiff[nturn]**2 +Ydiff[nturn]**2)
    Adiff[nturn] = np.log10(Adiff[nturn]+1.0e-6)
    Xdiff[nturn] = np.log10(Xdiff[nturn]+1.0e-6)
    Ydiff[nturn] = np.log10(Ydiff[nturn]+1.0e-6)
    iMax[nturn] = np.argmax(Adiff[nturn]),np.argmax(Xdiff[nturn]),np.argmax(Ydiff[nturn])
    iMin[nturn] = np.argmin(Adiff[nturn]),np.argmin(Xdiff[nturn]),np.argmin(Ydiff[nturn])

### Plot

##### FFT

In [ ]:
n=NTURN[-1]
n0=NTURN[-2]
plt.figure(figsize=[8,6])
plt.subplot(3,2,1)
plt.semilogy(np.abs(np.fft.fft(X[n][:,iMax[n][1]])),'r:')
plt.semilogy(np.abs(np.fft.fft(X[n][:,iMin[n][1]])),'k')
plt.xlabel('fft$_x$')
plt.subplot(3,2,2)
plt.semilogy(np.abs(np.fft.fft(Y[n][:,iMax[n][2]])),'r:')
plt.semilogy(np.abs(np.fft.fft(Y[n][:,iMin[n][2]])),'k')
plt.legend(['most chaotic','most regular'],loc=1)
plt.xlabel('fft$_y$')
plt.subplot(3,2,3)
plt.semilogy(np.abs(np.fft.fft(X[n][:n0,iMin[n][1]])),'r:')
plt.semilogy(np.abs(np.fft.fft(X[n][n0:,iMin[n][1]])),'k')
plt.xlabel('fft_regular_comparison$_x$')
plt.subplot(3,2,4)
plt.semilogy(np.abs(np.fft.fft(Y[n][:n0,iMin[n][2]])),'r:')
plt.semilogy(np.abs(np.fft.fft(Y[n][n0:,iMin[n][2]])),'k')
plt.xlabel('fft_regular_comparison$_y$')
plt.legend(['1st half','2nd half'],loc=1)
plt.subplot(3,2,5)
plt.semilogy(np.abs(np.fft.fft(X[n][:n0,iMax[n][1]])),'r:')
plt.semilogy(np.abs(np.fft.fft(X[n][n0:,iMax[n][1]])),'k')
plt.xlabel('fft_chotic_comparison$_x$')
plt.subplot(3,2,6)
plt.semilogy(np.abs(np.fft.fft(Y[n][:n0,iMax[n][2]])),'r:')
plt.semilogy(np.abs(np.fft.fft(Y[n][n0:,iMax[n][2]])),'k')
plt.xlabel('fft_chotic_comparison$_y$')
plt.legend(['1st half','2nd half'],loc=1)
plt.tight_layout()
plt.savefig('FMA.FFT.nu='+str(NL_nu)+'.tau='+str(NL_t)+'.mu='+str(dTune)+'.png',dpi=180)
# plt.close()

##### Tune Plot

In [ ]:
def resonnace(ix,iy,N):
    if ix==0 and iy==0:
        nux=0
        nuy=0
    else:
        if ix==0:
            nux = np.linspace(-1,1,2)
            nuy = np.linspace(float( N )/iy, float( N )/iy, 2)
        elif iy==0:
            nuy = np.linspace(-1,1,2)
            nux = np.linspace(float( N )/ix, float( N )/ix, 2)
        else:        
            nux = np.linspace(-1,1,2)
            nuy = ix/float(iy)*nux +  N/float(iy)
    return nux,nuy

In [ ]:
plt.figure(figsize=[4,3])
for i in range(-4,4):
    for j in range(-4,4):
        for k in range(-4,4):
            nux,nuy = resonnace(i,j,k)
            plt.plot(nux,nuy,c='k',lw=0.2,ls='--',alpha=0.7)
            
for i in range(-6,6):
    for j in range(-6,6):
        for k in range(-6,6):
            nux,nuy = resonnace(i,j,k)
            plt.plot(nux,nuy,c='k',lw=0.1,ls='--',alpha=0.7)
            
f1=plt.scatter(tuneXbefore[n],tuneYbefore[n],c=Adiff[n],s=1.0,marker='o')

tmp = np.sort(tuneXbefore[n])
std = 0.2*np.std(tmp)
xmin = tmp[32]-std
xmax = tmp[-32]+std

tmp = np.sort(tuneYbefore[n])
std = 0.2*np.std(tmp)
ymin = tmp[32]-std
ymax = tmp[-32]+std

plt.xlabel(r'$\nu_x$')
plt.ylabel(r'$\nu_y$')
plt.colorbar(f1)
plt.xlim(xmin,xmax)
plt.ylim(ymin,ymax)
plt.tight_layout()
plt.savefig('FMA.tune.nu='+str(NL_nu)+'.tau='+str(NL_t)+'.mu='+str(dTune)+'.png',dpi=180)
# plt.close()

In [ ]:
import scipy

pData = impact.readParticleData(111110,ke,mass,freq,format_id=2)[npt:,:4]
normalize(pData)
dist = scipy.stats.gaussian_kde(pData[:,[0,2]].transpose())

weight = dist([np.real(X[n][0,:]),np.real(Y[n][0,:])])

In [ ]:
iCore = np.argwhere(weight > 0.2) # empirically found ...
iCore = iTBT[n][iCore]

iCoreTmp = np.in1d(iTBT[n],iCore)
xTmp = np.real(X[n][0,iCoreTmp])
yTmp = np.real(Y[n][0,iCoreTmp])

grid = impact.data.dictClass()
grid.extent = [xTmp.min(),xTmp.max(),yTmp.min(),yTmp.max()]
grid.x, grid.y = np.mgrid[xTmp.min():xTmp.max():64j, yTmp.min():yTmp.max():64j]
grid.points = np.vstack([grid.x.ravel(), grid.y.ravel()])
grid.z = np.reshape(dist(grid.points).T, grid.x.shape)

plt.figure(figsize=(12,3.3))
plt.subplot(1,3,1)
plt.scatter(np.real(X[n][0,:]),np.real(Y[n][0,:]),s=0.4)
plt.scatter(pData[::10,0],pData[::10,2],s=0.1)
plt.scatter(xTmp,yTmp,s=4.0)
plt.subplot(1,3,2)
plt.scatter(pData[::10,0],pData[::10,2],s=0.1)
plt.scatter(xTmp,yTmp,s=4.0)
plt.subplot(1,3,3)
plt.scatter(pData[::10,0],pData[::10,2],s=0.1)
plt.contour(grid.z.T, extent=grid.extent, levels=[0.2]);
plt.tight_layout()

pData = None
dist = None
weight = None

In [ ]:
plt.figure(figsize=[4,3])
for i in range(-4,4):
    for j in range(-4,4):
        for k in range(-4,4):
            nux,nuy = resonnace(i,j,k)
            plt.plot(nux,nuy,c='k',lw=0.2,ls='--',alpha=0.7)
            
for i in range(-6,6):
    for j in range(-6,6):
        for k in range(-6,6):
            nux,nuy = resonnace(i,j,k)
            plt.plot(nux,nuy,c='k',lw=0.1,ls='--',alpha=0.7)
            
            
f1=plt.scatter(tuneXbefore[n][iCoreTmp],tuneYbefore[n][iCoreTmp],c=Adiff[n][iCoreTmp],s=1.0,marker='o')

tmp = np.sort(tuneXbefore[n][iCoreTmp])
std = 0.2*np.std(tmp)
xmin = tmp[8]-std
xmax = tmp[-8]+std

tmp = np.sort(tuneYbefore[n][iCoreTmp])
std = 0.2*np.std(tmp)
ymin = tmp[8]-std
ymax = tmp[-8]+std

plt.xlabel(r'$\nu_x$')
plt.ylabel(r'$\nu_y$')
plt.colorbar(f1)
plt.xlim(xmin,xmax)
plt.ylim(ymin,ymax)
plt.tight_layout()
plt.savefig('FMA.tune.core.nu='+str(NL_nu)+'.tau='+str(NL_t)+'.mu='+str(dTune)+'.png',dpi=180)
# plt.close()

In [ ]:
r0_in = 0.7
Arc0yIn = np.linspace(-r0_in,r0_in,100)
Arc0xIn = -np.sqrt(r0_in**2-Arc0yIn**2)
x0in_origin = 1.01


r0_out = 0.7
Arc0yOut = np.linspace(-r0_out,r0_out,100)
Arc0xOut = -np.sqrt(r0_out**2-Arc0yOut**2)
x0out_origin = 1.11

pInArc = np.logical_and( ((np.real(X[n][0,:])-x0in_origin )**2 + np.real(Y[n][0,:])**2) > r0_in**2  ,
                         ((np.real(X[n][0,:])+x0in_origin )**2 + np.real(Y[n][0,:])**2) > r0_in**2  )
pOutArc= np.logical_or(  ((np.real(X[n][0,:])-x0out_origin)**2 + np.real(Y[n][0,:])**2) < r0_out**2 ,
                         ((np.real(X[n][0,:])+x0out_origin)**2 + np.real(Y[n][0,:])**2) < r0_out**2 )
pOnArc = np.logical_not( np.logical_or( pInArc, pOutArc))

In [ ]:
xmax = np.max([np.real(X[n][0,:]),np.real(Y[n][0,:])])
plt.figure(figsize=[3.7,3.3])
plt.plot(Arc0xIn   +x0in_origin, Arc0yIn,c='r')
plt.plot(Arc0xOut  +x0out_origin,Arc0yOut,c='r')
plt.plot(-Arc0xIn  -x0in_origin, Arc0yIn,c='r')
plt.plot(-Arc0xOut -x0out_origin,Arc0yOut,c='r')
plt.scatter(np.real(X[n][0,pInArc]),np.real(Y[n][0,pInArc]),s=1.6,marker='s')
plt.scatter(np.real(X[n][0,pOutArc]),np.real(Y[n][0,pOutArc]),s=1.6,marker='s')
plt.scatter(np.real(X[n][0,pOnArc]),np.real(Y[n][0,pOnArc]),s=1.6,marker='s')
plt.xlim(-xmax*1.1,xmax*1.1)
plt.ylim(-xmax*1.1,xmax*1.1)
plt.xlabel('$x_n$')
plt.ylabel('$y_n$')
plt.tight_layout()
plt.savefig('geometry.nu='+str(NL_nu)+'.tau='+str(NL_t)+'.mu='+str(dTune)+'.png',dpi=180)

In [ ]:
plt.figure(figsize=[4,3])
for i in range(-4,4):
    for j in range(-4,4):
        for k in range(-4,4):
            nux,nuy = resonnace(i,j,k)
            plt.plot(nux,nuy,c='k',lw=0.2,ls='--',alpha=0.7)
            
for i in range(-6,6):
    for j in range(-6,6):
        for k in range(-6,6):
            nux,nuy = resonnace(i,j,k)
            plt.plot(nux,nuy,c='k',lw=0.1,ls='--',alpha=0.7)

            
f1=plt.scatter(tuneXbefore[n][pInArc],tuneYbefore[n][pInArc],c=Adiff[n][pInArc],s=1.0,marker='o')

if NL_nu==0.28:
    nux,nuy = resonnace(3,0,1)
    plt.plot(nux,nuy,c='r',lw=0.8,ls='--')
    plt.text(0.34, 0.07, r'$3\nu_x=1$', fontsize=11, color='r', rotation=90, rotation_mode='anchor')
    nux,nuy = resonnace(1,2,0)
    plt.plot(nux,nuy,c='r',lw=0.8,ls='--')
    plt.text(0.362, 0.188, r'$2\nu_y-\nu_x=0$', fontsize=11, color='r', rotation=15, rotation_mode='anchor')
    nux,nuy = resonnace(-2,2,1)
    plt.plot(nux,nuy,c='r',lw=0.8,ls='--')
    plt.text(0.301, 0.187, r'$2\nu_x+2\nu_y=1$', fontsize=11, color='r', rotation=-30, rotation_mode='anchor')


tmp = np.sort(tuneXbefore[n][pInArc])
std = 0.2*np.std(tmp)
xmin = tmp[8]-std
xmax = tmp[-8]+std

tmp = np.sort(tuneYbefore[n][pInArc])
std = 0.2*np.std(tmp)
ymin = tmp[8]-std
ymax = tmp[-8]+std

plt.xlabel(r'$\nu_x$')
plt.ylabel(r'$\nu_y$')
plt.colorbar(f1)
plt.xlim(xmin,xmax)
plt.ylim(ymin,ymax)
plt.tight_layout()
plt.savefig('FMA.tune.regionI.nu='+str(NL_nu)+'.tau='+str(NL_t)+'.mu='+str(dTune)+'.png',dpi=180)

In [ ]:
plt.figure(figsize=[4,3])
for i in range(-4,4):
    for j in range(-4,4):
        for k in range(-4,4):
            nux,nuy = resonnace(i,j,k)
            plt.plot(nux,nuy,c='k',lw=0.2,ls='--',alpha=0.7)
            
for i in range(-6,6):
    for j in range(-6,6):
        for k in range(-6,6):
            nux,nuy = resonnace(i,j,k)
            plt.plot(nux,nuy,c='k',lw=0.1,ls='--',alpha=0.7)
            
            
f1=plt.scatter(tuneXbefore[n][pOutArc],tuneYbefore[n][pOutArc],c=Adiff[n][pOutArc],s=1.0,marker='o')

tmp = np.sort(tuneXbefore[n][pOutArc])
std = 0.2*np.std(tmp)
xmin = tmp[8]-std
xmax = tmp[-8]+std

tmp = np.sort(tuneYbefore[n][pOutArc])
std = 0.2*np.std(tmp)
ymin = tmp[8]-std
ymax = tmp[-8]+std

plt.xlabel(r'$\nu_x$')
plt.ylabel(r'$\nu_y$')
plt.colorbar(f1)
plt.xlim(xmin,xmax)
plt.ylim(ymin,ymax)
plt.tight_layout()
plt.savefig('FMA.tune.regionII.nu='+str(NL_nu)+'.tau='+str(NL_t)+'.mu='+str(dTune)+'.png',dpi=180)

In [ ]:
plt.figure(figsize=[4,3])
for i in range(-4,4):
    for j in range(-4,4):
        for k in range(-4,4):
            nux,nuy = resonnace(i,j,k)
            plt.plot(nux,nuy,c='k',lw=0.2,ls='--',alpha=0.7)
            
for i in range(-6,6):
    for j in range(-6,6):
        for k in range(-6,6):
            nux,nuy = resonnace(i,j,k)
            plt.plot(nux,nuy,c='k',lw=0.1,ls='--',alpha=0.7)
            
            
f1=plt.scatter(tuneXbefore[n][pOnArc],tuneYbefore[n][pOnArc],c=Adiff[n][pOnArc],s=1.0,marker='o')

tmp = np.sort(tuneXbefore[n][pOnArc])
std = 0.2*np.std(tmp)
xmin = tmp[8]-std
xmax = tmp[-8]+std

tmp = np.sort(tuneYbefore[n][pOnArc])
std = 0.2*np.std(tmp)
ymin = tmp[8]-std
ymax = tmp[-8]+std

plt.xlabel(r'$\nu_x$')
plt.ylabel(r'$\nu_y$')
plt.colorbar(f1)
plt.xlim(xmin,xmax)
plt.ylim(ymin,ymax)
plt.tight_layout()
plt.savefig('FMA.tune.regionIII.nu='+str(NL_nu)+'.tau='+str(NL_t)+'.mu='+str(dTune)+'.png',dpi=180)

In [ ]:
plt.figure(figsize=[4,3])
for i in range(-4,4):
    for j in range(-4,4):
        for k in range(-4,4):
            nux,nuy = resonnace(i,j,k)
            plt.plot(nux,nuy,c='k',lw=0.2,ls='--',alpha=0.7)
            
for i in range(-6,6):
    for j in range(-6,6):
        for k in range(-6,6):
            nux,nuy = resonnace(i,j,k)
            plt.plot(nux,nuy,c='k',lw=0.1,ls='--',alpha=0.7)
            
            
plt.scatter(tuneXbefore[n][pInArc],tuneYbefore[n][pInArc],s=0.5,marker='o',alpha=0.8)
plt.scatter(tuneXbefore[n][pOutArc],tuneYbefore[n][pOutArc],s=0.5,marker='o',alpha=0.8)
plt.scatter(tuneXbefore[n][pOnArc],tuneYbefore[n][pOnArc],s=0.5,marker='o',alpha=0.8)

tmp = np.sort(tuneXbefore[n])
std = 0.2*np.std(tmp)
xmin = tmp[8]-std
xmax = tmp[-8]+std

tmp = np.sort(tuneYbefore[n])
std = 0.2*np.std(tmp)
ymin = tmp[8]-std
ymax = tmp[-8]+std

plt.xlabel(r'$\nu_x$')
plt.ylabel(r'$\nu_y$')
plt.xlim(xmin,xmax)
plt.ylim(ymin,ymax)
plt.tight_layout()
plt.savefig('FMA.tune.regions.nu='+str(NL_nu)+'.tau='+str(NL_t)+'.mu='+str(dTune)+'.png',dpi=180)

##### Poincare

In [ ]:
plt.figure(figsize=[3.3,3])
plt.scatter(np.real(X[n][:,iMax[n][0]]),np.real(Y[n][:,iMax[n][0]]),c='r',s=0.1)
plt.scatter(np.real(X[n][:,iMin[n][0]]),np.real(Y[n][:,iMin[n][0]]),c='k',s=0.1)
plt.xlabel('x')
plt.ylabel('y')
plt.legend(['most chaotic','most regular'],loc=1)
plt.tight_layout()
plt.savefig('FMA.poincare_xy.nu='+str(NL_nu)+'.tau='+str(NL_t)+'.mu='+str(dTune)+'.png',dpi=180)

In [ ]:
plt.figure(figsize=[3.3,3])
plt.scatter(np.real(X[n][:,iMax[n][0]]),np.imag(X[n][:,iMax[n][0]]),c='r',s=0.1)
plt.scatter(np.real(X[n][:,iMin[n][0]]),np.imag(X[n][:,iMin[n][0]]),c='k',s=0.1)
plt.xlabel('x')
plt.ylabel('px')
plt.legend(['most chaotic','most regular'],loc=1)
plt.tight_layout()
plt.savefig('FMA.poincare_xpx.nu='+str(NL_nu)+'.tau='+str(NL_t)+'.mu='+str(dTune)+'.png',dpi=180)

In [ ]:
plt.figure(figsize=[3.3,3])
plt.scatter(np.real(Y[n][:,iMax[n][0]]),np.imag(Y[n][:,iMax[n][0]]),c='r',s=0.1)
plt.scatter(np.real(Y[n][:,iMin[n][0]]),np.imag(Y[n][:,iMin[n][0]]),c='k',s=0.1)
plt.xlabel('y')
plt.ylabel('py')
plt.legend(['most chaotic','most regular'],loc=1)
plt.tight_layout()
plt.savefig('FMA.poincare_ypy.nu='+str(NL_nu)+'.tau='+str(NL_t)+'.mu='+str(dTune)+'.png',dpi=180)

##### DA

In [ ]:
n=NTURN[0]
xmax = np.max([np.real(X[n][0,:]),np.real(Y[n][0,:])])
for n in NTURN[1:]:
    plt.figure(figsize=[4,3.3])
    f1=plt.scatter(np.real(X[n][0,:]),np.real(Y[n][0,:]),c=Adiff[n],s=1.6,marker='s')
    plt.scatter([-1.0,1.0],[0.0,0.0],c='r',s=77,marker='x')
    plt.xlim(-xmax*1.1,xmax*1.1)
    plt.ylim(-xmax*1.1,xmax*1.1)
    plt.xlabel('$x_n$')
    plt.ylabel('$y_n$')
    plt.colorbar(f1)
    plt.tight_layout()
    plt.savefig('FMA.DA.nu='+str(NL_nu)+'.tau='+str(NL_t)+'.mu='+str(dTune)+'.turn='+str(int(n/2))+'.png',dpi=180)

# FB

### Run Backward

lattice.pop()
latticeB = copy(lattice[1:][::-1])
latticeB.insert(0,copy(lattice[0]))

for i in range(len(latticeB)):
    if 'length' in latticeB[i]:
        latticeB[i].length = -latticeB[i].length
    if latticeB[i].type == 'linear_matrix_map':
        latticeB[i].tune_advance                 =-latticeB[i].tune_advance
        latticeB[i].nonlinear_insert_tuneAdvance =-latticeB[i].nonlinear_insert_tuneAdvance
        latticeB[i].nonlinear_insert_length      =-latticeB[i].nonlinear_insert_length

for nturn in NTURN[:-1]:
    latticeB[0].turns = nturn
    latticeB[-1].file_id = 100000+nturn
    beam.distribution.distribution_type = 'ReadFile_binary'
    beam.distribution.file_id = nturn
    impact.writeInputFile(beam,latticeB) 
    impact.run(beam)

### Load Data

for nturn in NTURN[:-1]:
    print(nturn)
    iTBT[-nturn],TBT[-nturn] = impact.readTBT(100000+nturn,ke,mass,freq)
    TBT[-nturn] = TBT[-nturn][:,:4,:]
    Npt[-nturn] = len(iTBT[-nturn])

for i in range(len(NTURN)-1):
    nturn = NTURN[i]
    
    npt = len(iTBT[-nturn])
    X[nturn]=np.zeros((nturn,npt),dtype=np.complex)
    Y[nturn]=np.zeros((nturn,npt),dtype=np.complex)
    tmp = TBT[-nturn][::-1,:4,:]
    X[-nturn]=(tmp[:,0,:] - 1j*(betx*tmp[:,1,:]+alfx*tmp[:,0,:]))/(NL_c*np.sqrt(betx))
    Y[-nturn]=(tmp[:,2,:] - 1j*(betx*tmp[:,3,:]+alfx*tmp[:,2,:]))/(NL_c*np.sqrt(betx))
    
    iturn = 0
    for j in range(i+1):
        jturn = NTURN[j]
        
        itmp = np.in1d(iTBT[jturn],iTBT[-nturn])
        tmp = TBT[jturn][:,:,itmp]
        X[nturn][iturn:jturn] = (tmp[:,0,:] - 1j*(betx*tmp[:,1,:]+alfx*tmp[:,0,:]))/(NL_c*np.sqrt(betx))
        Y[nturn][iturn:jturn] = (tmp[:,2,:] - 1j*(betx*tmp[:,3,:]+alfx*tmp[:,2,:]))/(NL_c*np.sqrt(betx))
        print(iturn,jturn,nturn,np.sum(itmp),npt)
        
        iturn = jturn

for nturn in NTURN[:-1]:
    nturn = -nturn
    Xdiff[nturn] = np.abs(X[nturn]-X[-nturn])**2
    Ydiff[nturn] = np.abs(Y[nturn]-Y[-nturn])**2
    Adiff[nturn] = Xdiff[nturn] + Ydiff[nturn]
    Xdiff[nturn] = np.sqrt(-np.sum(Xdiff[nturn],0)/nturn)
    Ydiff[nturn] = np.sqrt(-np.sum(Ydiff[nturn],0)/nturn)
    Adiff[nturn] = np.sqrt(-np.sum(Adiff[nturn],0)/nturn)
    Xdiff[nturn] = np.log10(Xdiff[nturn])
    Ydiff[nturn] = np.log10(Ydiff[nturn])
    Adiff[nturn] = np.log10(Adiff[nturn])
    iMax[nturn] = np.argmax(Adiff[nturn]),np.argmax(Xdiff[nturn]),np.argmax(Ydiff[nturn])
    iMin[nturn] = np.argmin(Adiff[nturn]),np.argmin(Xdiff[nturn]),np.argmin(Ydiff[nturn])

### Plot

##### Poincare

n=-NTURN[-2]
plt.figure(figsize=[3.3,3])
plt.scatter(np.real(X[n][:,iMax[n][0]]),np.real(Y[n][:,iMax[n][0]]),c='r',s=0.1)
plt.scatter(np.real(X[n][:,iMin[n][0]]),np.real(Y[n][:,iMin[n][0]]),c='k',s=0.1)
plt.xlabel('x')
plt.ylabel('y')
plt.legend(['most chaotic','most regular'],loc=1)
plt.tight_layout()
plt.savefig('FB.poincare_xy.nu='+str(NL_nu)+'.tau='+str(NL_t)+'.mu='+str(dTune)+'.png',dpi=180)

plt.figure(figsize=[3.3,3])
plt.scatter(np.real(X[n][:,iMax[n][0]]),np.imag(X[n][:,iMax[n][0]]),c='r',s=0.1)
plt.scatter(np.real(X[n][:,iMin[n][0]]),np.imag(X[n][:,iMin[n][0]]),c='k',s=0.1)
plt.xlabel('x')
plt.ylabel('px')
plt.legend(['most chaotic','most regular'],loc=1)
plt.tight_layout()
plt.savefig('FB.poincare_xpx.nu='+str(NL_nu)+'.tau='+str(NL_t)+'.mu='+str(dTune)+'.png',dpi=180)

plt.figure(figsize=[3.3,3])
plt.scatter(np.real(Y[n][:,iMax[n][0]]),np.imag(Y[n][:,iMax[n][0]]),c='r',s=0.1)
plt.scatter(np.real(Y[n][:,iMin[n][0]]),np.imag(Y[n][:,iMin[n][0]]),c='k',s=0.1)
plt.xlabel('y')
plt.ylabel('py')
plt.legend(['most chaotic','most regular'],loc=1)
plt.tight_layout()
plt.savefig('FB.poincare_ypy.nu='+str(NL_nu)+'.tau='+str(NL_t)+'.mu='+str(dTune)+'.png',dpi=180)

##### DA

n=NTURN[0]
xmax = np.max([np.real(X[n][0,:]),np.real(Y[n][0,:])])
for n in NTURN[:-1]:
    plt.figure(figsize=[4,3.3])
    f1=plt.scatter(np.real(X[n][0,:]),np.real(Y[n][0,:]),c=Adiff[-n],s=1.6,marker='s')
    plt.scatter([-1.0,1.0],[0.0,0.0],c='r',s=77,marker='x')
    plt.xlim(-xmax*1.1,xmax*1.1)
    plt.ylim(-xmax*1.1,xmax*1.1)
    plt.xlabel('$x_n$')
    plt.ylabel('$y_n$')
    plt.colorbar(f1)
    plt.tight_layout()
    plt.savefig('FB.DA.nu='+str(NL_nu)+'.tau='+str(NL_t)+'.mu='+str(dTune)+'.turn='+str(n)+'.png',dpi=180)